In [13]:
from transformers import BertTokenizer, TFBertModel,BertForQuestionAnswering,BertForMaskedLM,BertModel
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import pipeline
import logging
import tensorflow as tf
import torch
from nltk.tokenize import sent_tokenize
import faiss
import numpy as np

logging.getLogger("transformers.modeling_utils").setLevel(logging.WARNING)

In [ ]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

file_path = '/Users/t-arvio.anandi/Downloads/Train Label - Sheet1 (2).csv'
df = pd.read_csv(file_path)

x_data_list = df['x_data'].tolist()
y_data_list = df['y_data'].tolist()

encoding = tokenizer(x_data_list, padding='max_length', truncation=True, return_tensors='tf', max_length=40)
x_input_ids = encoding['input_ids']
x_attention_mask = encoding['attention_mask']

x_embeddings = bert_model(x_input_ids, attention_mask=x_attention_mask)[0].numpy() 

target_size = 50

if x_embeddings.shape[2] < target_size:
    padding = target_size - x_embeddings.shape[2]
    x_embeddings_padded = np.pad(x_embeddings, ((0, 0), (0, 0), (0, padding)), 'constant')
else:
    x_embeddings_padded = x_embeddings[:, :, :target_size]


def clean_label(label):
    if pd.isna(label):
        return [0]  
    try:
        return list(map(int, label.split()))
    except:
        return [0]  

y_sequences = [clean_label(label) for label in y_data_list]

y_sequences_padded = pad_sequences(y_sequences, maxlen=40, padding='post', truncating='post', value=0)  # Using -1 or another placeholder for padding

print("Padded X Shape:", x_embeddings.shape)
print("Padded Y Shape:", y_sequences_padded.shape)

assert x_embeddings.shape[0] == 91 
assert x_embeddings.shape[1] == 40 
print(x_embeddings.shape[2]) 
assert y_sequences_padded.shape[0] == 91
assert y_sequences_padded.shape[1] == 40
print(y_sequences_padded.shape[1])


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

inputs = layers.Input(shape=(40, 768)) 

lstm_out = layers.LSTM(64, return_sequences=True)(inputs)

outputs = layers.TimeDistributed(layers.Dense(7, activation='softmax'))(lstm_out)

model = models.Model(inputs, outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(model.summary())


In [ ]:
print(y_sequences_padded)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(x_input_ids, y_sequences_padded, test_size=0.2)

model.fit(x_input_ids, y_sequences_padded, epochs=3, batch_size=32)

In [ ]:

x_input_ids_np = np.array(x_input_ids)
y_labels_np = np.array(y_labels)

print("Shape of x_input_ids:", x_input_ids_np.shape)
print("Shape of y_labels:", y_labels_np.shape)

In [ ]:
new_sentence = "What is your name?"
new_tokens = tokenizer.tokenize(new_sentence)
new_input_ids = tokenizer.convert_tokens_to_ids(new_tokens)
new_input_ids = pad_sequences([new_input_ids], maxlen=max_len, padding='post', truncating='post')

predictions = model.predict(new_input_ids)
predicted_labels = tf.argmax(predictions, axis=-1)

print("Predicted Labels:", predicted_labels)